In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np

In [4]:
# project_name = 'Split_Sys_VQA'
project_name = 'Split_Sub_VQA'

# Show Novel and Non-novel Testing Results

In [ ]:

exp_name = 'Naive-lr0_005'       # nc-
# exp_name = 'nc-LwF'
# exp_name = 'nc-LwF'
# exp_name = 'GEM'

test_case_name = 'Novel-{}'.format(exp_name)
# test_case_name = 'Novel-{}-frz'.format(exp_name)
# test_case_name = 'Novel-{}-lm'.format(exp_name)
# test_case_name = 'Novel-{}-frz-lm'.format(exp_name)

In [31]:
results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{test_case_name}.npy'), allow_pickle=True)
print(results)

[{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.6334140435835351, 'Loss_Epoch/train_phase/train_stream/Task000': 0.6589970446094763, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.5, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.7359500527381897, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.5, 'Loss_Stream/eval_phase/test_stream/Task000': 0.7359500527381897, 'StreamForgetting/eval_phase/test_stream': 0.0}
 {'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.8539196940726578, 'Loss_Epoch/train_phase/train_stream/Task000': 0.3170702129830366, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.0, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 20.3140230178833, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.4, 'Loss_Stream/eval_phase/test_stream/Task000': 10.416035443544388, 'StreamForgetting/eval_phase/test_stream': 0.5, 'ExperienceForgetting/eval_phase/test_stream/Task000/Exp000': 0.5, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.8, 

In [32]:
'''Test acc'''
accs = [result['Top1_Acc_Stream/eval_phase/test_stream/Task000'] for result in results]
print(accs)

[0.5, 0.4, 0.2966666666666667, 0.1775, 0.17, 0.15833333333333333, 0.12, 0.09625, 0.08, 0.078]


In [33]:
print(f'{np.mean(accs)*100:.2f}% ({np.std(accs)*100:.2f}%)')

20.77% (13.72%)


# Show Hyper-Parameter Tuning Results

In [3]:
def display_results(exp_name, test_case_name):
    print(f'{exp_name}, {test_case_name}:')
    results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{test_case_name}.npy'), allow_pickle=True)
    accs = [result['Top1_Acc_Stream/eval_phase/test_stream/Task000'] for result in results]
    print(f'==> {accs}')
    print(f'==> {np.mean(accs)*100:.2f}% ({np.std(accs)*100:.2f}%)')

In [5]:
exp_names = [
    'Naive-lr0_001', 'Naive-lr0_005', 'Naive-lr0_01', 'Naive-lr0_05', 'Naive-lr0_1',
]
exp_names = [f'color-{exp_names[i]}' for i in range(len(exp_names))]

for exp_name in exp_names:
    display_results(exp_name, exp_name)

color-Naive-lr0_001, color-Naive-lr0_001:
==> [0.97, 0.48, 0.30333333333333334, 0.2375, 0.178, 0.16, 0.13428571428571429, 0.10625, 0.10333333333333333, 0.092]
==> 27.65% (25.71%)
color-Naive-lr0_005, color-Naive-lr0_005:
==> [0.98, 0.475, 0.3, 0.24, 0.174, 0.15, 0.14, 0.11125, 0.10666666666666667, 0.093]
==> 27.70% (25.91%)
color-Naive-lr0_01, color-Naive-lr0_01:
==> [0.96, 0.465, 0.30666666666666664, 0.2425, 0.172, 0.145, 0.13714285714285715, 0.115, 0.10333333333333333, 0.091]
==> 27.38% (25.35%)
color-Naive-lr0_05, color-Naive-lr0_05:
==> [0.95, 0.47, 0.30333333333333334, 0.245, 0.168, 0.16, 0.13857142857142857, 0.10875, 0.09888888888888889, 0.092]
==> 27.35% (25.11%)
color-Naive-lr0_1, color-Naive-lr0_1:
==> [0.98, 0.47, 0.31, 0.2375, 0.162, 0.15333333333333332, 0.14, 0.1075, 0.09888888888888889, 0.091]
==> 27.50% (26.01%)


In [42]:
exp_names = [
    'LwF-lr0_001-m100', 'LwF-lr0_01-m100', 'LwF-lr0_1-m100',
    'LwF-lr0_001-m500', 'LwF-lr0_01-m500', 'LwF-lr0_1-m500',
    'LwF-lr0_001-m1000', 'LwF-lr0_01-m1000', 'LwF-lr0_1-m1000',
]
exp_names = [f'color-{exp_names[i]}' for i in range(len(exp_names))]

for exp_name in exp_names:
    display_results(exp_name, exp_name)

ER-lr0_001-m100, ER-lr0_001-m100:
==> [0.64, 0.4, 0.29333333333333333, 0.2375, 0.182, 0.165, 0.12714285714285714, 0.115, 0.1, 0.082]
==> 23.42% (16.43%)
ER-lr0_01-m100, ER-lr0_01-m100:
==> [0.66, 0.415, 0.2866666666666667, 0.25, 0.174, 0.16333333333333333, 0.13142857142857142, 0.10625, 0.09666666666666666, 0.085]
==> 23.68% (17.14%)
ER-lr0_1-m100, ER-lr0_1-m100:
==> [0.5, 0.41, 0.29333333333333333, 0.225, 0.176, 0.16333333333333333, 0.12285714285714286, 0.1175, 0.09111111111111111, 0.085]
==> 21.84% (13.41%)
ER-lr0_001-m500, ER-lr0_001-m500:
==> [0.64, 0.525, 0.33666666666666667, 0.2375, 0.23, 0.2, 0.1742857142857143, 0.145, 0.13, 0.116]
==> 27.34% (16.79%)
ER-lr0_01-m500, ER-lr0_01-m500:
==> [0.66, 0.6, 0.35333333333333333, 0.32, 0.222, 0.205, 0.19, 0.18, 0.17, 0.13]
==> 30.30% (17.61%)
ER-lr0_1-m500, ER-lr0_1-m500:
==> [0.5, 0.54, 0.4, 0.245, 0.274, 0.195, 0.17714285714285713, 0.15, 0.10444444444444445, 0.108]
==> 26.94% (15.02%)
ER-lr0_001-m1000, ER-lr0_001-m1000:
==> [0.64, 0.45, 0

In [6]:
exp_names = [
    'LwF-lr0_001-a10-t2', 'LwF-lr0_005-a10-t2', 'LwF-lr0_01-a10-t2', 'LwF-lr0_05-a10-t2',  'LwF-lr0_1-a10-t2',
]
exp_names = [f'color-{exp_names[i]}' for i in range(len(exp_names))]

for exp_name in exp_names:
    display_results(exp_name, exp_name)

color-LwF-lr0_001-a10-t2, color-LwF-lr0_001-a10-t2:
==> [0.97, 0.445, 0.3, 0.2425, 0.168, 0.16333333333333333, 0.14, 0.10375, 0.10222222222222223, 0.092]
==> 27.27% (25.46%)
color-LwF-lr0_005-a10-t2, color-LwF-lr0_005-a10-t2:
==> [0.98, 0.455, 0.31, 0.2375, 0.178, 0.16, 0.14, 0.11, 0.10555555555555556, 0.094]
==> 27.70% (25.72%)
color-LwF-lr0_01-a10-t2, color-LwF-lr0_01-a10-t2:
==> [0.96, 0.47, 0.30333333333333334, 0.2375, 0.168, 0.15166666666666667, 0.13714285714285715, 0.105, 0.10222222222222223, 0.091]
==> 27.26% (25.45%)
color-LwF-lr0_05-a10-t2, color-LwF-lr0_05-a10-t2:
==> [0.95, 0.445, 0.2833333333333333, 0.235, 0.136, 0.15, 0.1357142857142857, 0.11, 0.10222222222222223, 0.09]
==> 26.37% (25.11%)
color-LwF-lr0_1-a10-t2, color-LwF-lr0_1-a10-t2:
==> [0.98, 0.43, 0.2966666666666667, 0.24, 0.154, 0.16, 0.1357142857142857, 0.105, 0.08888888888888889, 0.09]
==> 26.80% (25.83%)


In [45]:
exp_names = [
    'GEM-lr0_01-p16-m0_3', 'GEM-lr0_01-p32-m0_3',
    'GEM-lr0_01-p16-m0_5', # 'GEM-lr0_01-p32-m0_5',

    'GEM-lr0_1-p16-m0_3', 'GEM-lr0_1-p32-m0_3',
    'GEM-lr0_1-p16-m0_5', # 'GEM-lr0_1-p32-m0_5',
]
exp_names = [f'color-{exp_names[i]}' for i in range(len(exp_names))]

for exp_name in exp_names:
    display_results(exp_name, exp_name)

GEM-lr0_01-p16-m0_3, GEM-lr0_01-p16-m0_3:
==> [0.66, 0.36, 0.2866666666666667, 0.145, 0.18, 0.14833333333333334, 0.16428571428571428, 0.11, 0.10111111111111111, 0.093]
==> 22.48% (16.59%)
GEM-lr0_01-p32-m0_3, GEM-lr0_01-p32-m0_3:
==> [0.66, 0.35, 0.28, 0.205, 0.186, 0.17666666666666667, 0.11857142857142858, 0.105, 0.08666666666666667, 0.087]
==> 22.55% (16.61%)
GEM-lr0_01-p16-m0_5, GEM-lr0_01-p16-m0_5:
==> [0.66, 0.37, 0.26666666666666666, 0.1975, 0.186, 0.13666666666666666, 0.13, 0.1225, 0.08333333333333333, 0.08]
==> 22.33% (16.81%)
GEM-lr0_1-p16-m0_3, GEM-lr0_1-p16-m0_3:
==> [0.5, 0.28, 0.27666666666666667, 0.1975, 0.148, 0.16166666666666665, 0.14, 0.10375, 0.07444444444444444, 0.086]
==> 19.68% (12.16%)
GEM-lr0_1-p32-m0_3, GEM-lr0_1-p32-m0_3:
==> [0.5, 0.395, 0.2866666666666667, 0.24, 0.164, 0.16666666666666666, 0.16285714285714287, 0.11625, 0.09888888888888889, 0.088]
==> 22.18% (12.88%)
GEM-lr0_1-p16-m0_5, GEM-lr0_1-p16-m0_5:
==> [0.5, 0.37, 0.2866666666666667, 0.215, 0.172, 0.16